<a href="https://colab.research.google.com/github/eea2003/ENS491/blob/main/Wanted_Input_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import json
import folium

# -------------------------
# INPUT HELPERS
# -------------------------

def ask_int(prompt):
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("❌ Please enter a valid integer.")

def ask_float(prompt):
    while True:
        try:
            return float(input(prompt))
        except ValueError:
            print("❌ Please enter a valid number.")

def ask_yes_no(prompt):
    while True:
        ans = input(prompt + " (y/n): ").lower()
        if ans in ["y", "yes"]: return True
        if ans in ["n", "no"]: return False
        print("❌ Please answer with y/n.")


# -------------------------
# INPUT FUNCTIONS
# -------------------------

def input_locations():
    locations = []
    n = ask_int("How many locations do you want to add?: ")

    for i in range(n):
        print(f"\n📍 LOCATION {i+1}")
        loc_id = input("Location ID (e.g., Loc0001): ")
        x = ask_float("X coordinate: ")
        y = ask_float("Y coordinate: ")
        z = ask_float("Z coordinate: ")

        locations.append({
            "Location ID": loc_id,
            "X": x,
            "Y": y,
            "Z": z
        })

    return locations


def input_loading_zones():
    loading = []
    n = ask_int("How many loading zones?: ")

    for i in range(n):
        print(f"\n⛏️ LOADING ZONE {i+1}")
        lid = input("Loading Zone ID: ")
        loc = input("Location ID: ")
        ton = ask_float("Tonnage/min: ")

        loading.append({
            "Loading Zone ID": lid,
            "Location ID": loc,
            "Tonnage/min": ton
        })

    return loading


def input_dumping_zones():
    dumping = []
    n = ask_int("How many dumping zones?: ")

    for i in range(n):
        print(f"\n🗑️ DUMPING ZONE {i+1}")
        did = input("Dumping Zone ID: ")
        loc = input("Location ID: ")
        ton = ask_float("Tonnage/min: ")

        dumping.append({
            "Dumping Zone ID": did,
            "Location ID": loc,
            "Tonnage/min": ton
        })

    return dumping


def input_gas_stations():
    stations = []
    n = ask_int("How many GAS stations?: ")

    for i in range(n):
        print(f"\n⛽ GAS STATION {i+1}")
        sid = input("Gas Station ID: ")
        loc = input("Location ID: ")
        rate = ask_float("Service Rate (L/min): ")
        pumps = ask_int("Number of Pumps: ")

        stations.append({
            "Gas Station ID": sid,
            "Location ID": loc,
            "Service Rate (L/min)": rate,
            "Number of Pumps": pumps
        })

    return stations


def input_charge_stations():
    charge = []
    n = ask_int("How many ELECTRIC charge stations?: ")

    for i in range(n):
        print(f"\n🔌 CHARGE STATION {i+1}")
        eid = input("Charge Station ID: ")
        loc = input("Location ID: ")
        rate = ask_float("Service Rate (kW): ")
        chargers = ask_int("Number of Chargers: ")

        charge.append({
            "Charge Station ID": eid,
            "Location ID": loc,
            "Service Rate (kW)": rate,
            "Number of Chargers": chargers
        })

    return charge


def input_roads():
    roads = []
    n = ask_int("How many roads?: ")

    for i in range(n):
        print(f"\n🛣️ ROAD {i+1}")
        rid = input("Road ID: ")
        tail = input("Tail Location ID: ")
        head = input("Head Location ID: ")
        curvature = input("Curvature: ")
        grade = input("Grade: ")
        lanes = ask_int("Number of lanes: ")

        roads.append({
            "Road ID": rid,
            "Tail": tail,
            "Head": head,
            "Curvature": curvature,
            "Grade": grade,
            "Number of Lanes": lanes
        })

    return roads


# -------------------------
# MAIN PROGRAM (RETURNS DATA)
# -------------------------

def main():
    print("\n==============================")
    print(" 🚧 INTERACTIVE JSON CREATOR ")
    print("==============================")

    data = {}

    if ask_yes_no("Add locations?"):
        data["Location"] = input_locations()

    if ask_yes_no("Add loading zones?"):
        data["Loading"] = input_loading_zones()

    if ask_yes_no("Add dumping zones?"):
        data["Dumping"] = input_dumping_zones()

    if ask_yes_no("Add gas stations?"):
        data["Gas Station"] = input_gas_stations()

    if ask_yes_no("Add charge stations?"):
        data["Charge Station"] = input_charge_stations()

    if ask_yes_no("Add roads?"):
        data["Road"] = input_roads()

    return data


# -------------------------
# RUN MAIN → CREATE JSON → BUILD MAP
# -------------------------

# 1) Input al
data = main()

# 2) JSON'a kaydet
filename = input("\nEnter output filename (e.g., my_map_data.json): ")
with open(filename, "w") as f:
    json.dump(data, f, indent=4)

print(f"\n✅ JSON file saved: {filename}")

# 3) Harita oluştur
print("\n🗺️ Generating interactive map...")

# Prepare locations
locations = {loc["Location ID"]: loc for loc in data.get("Location", [])}

# If no locations, skip
if not locations:
    raise ValueError("❌ No locations defined — cannot draw map.")

# Normalize coordinates
xs = [loc["X"] for loc in locations.values()]
ys = [loc["Y"] for loc in locations.values()]

min_x, max_x = min(xs), max(xs)
min_y, max_y = min(ys), max(ys)

def norm_x(x): return (x - min_x) / (max_x - min_x + 1e-9) * 100
def norm_y(y): return (y - min_y) / (max_y - min_y + 1e-9) * 100

m = folium.Map(location=[50, 50], zoom_start=4, tiles=None)

def add_marker(loc_id, text, color):
    loc = locations.get(loc_id)
    if not loc: return
    folium.CircleMarker(
        location=[norm_y(loc["Y"]), norm_x(loc["X"])],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        popup=text
    ).add_to(m)

# Base nodes
for loc_id, loc in locations.items():
    folium.CircleMarker(
        location=[norm_y(loc["Y"]), norm_x(loc["X"])],
        radius=4,
        color="black",
        fill=True,
        fill_color="gray",
        popup=f"Location: {loc_id}"
    ).add_to(m)

# Special zones
for L in data.get("Loading", []): add_marker(L["Location ID"], f"Loading {L['Loading Zone ID']}", "green")
for D in data.get("Dumping", []): add_marker(D["Location ID"], f"Dumping {D['Dumping Zone ID']}", "red")
for G in data.get("Gas Station", []): add_marker(G["Location ID"], f"Gas {G['Gas Station ID']}", "blue")
for E in data.get("Charge Station", []): add_marker(E["Location ID"], f"Charge {E['Charge Station ID']}", "purple")

# Roads
for R in data.get("Road", []):
    t = locations.get(R["Tail"])
    h = locations.get(R["Head"])
    if t and h:
        folium.PolyLine(
            locations=[[norm_y(t["Y"]), norm_x(t["X"])],
                       [norm_y(h["Y"]), norm_x(h["X"])]],
            color="black",
            weight=2
        ).add_to(m)

m


 🚧 INTERACTIVE JSON CREATOR 
Add locations? (y/n): y
How many locations do you want to add?: 4

📍 LOCATION 1
Location ID (e.g., Loc0001): L0001
X coordinate: 23
Y coordinate: 67
Z coordinate: 2

📍 LOCATION 2
Location ID (e.g., Loc0001): L0002
X coordinate: 65
Y coordinate: 23
Z coordinate: 4

📍 LOCATION 3
Location ID (e.g., Loc0001): L0003
X coordinate: 34
Y coordinate: 87
Z coordinate: 5

📍 LOCATION 4
Location ID (e.g., Loc0001): L0004
X coordinate: 56
Y coordinate: 14
Z coordinate: 2
Add loading zones? (y/n): Y
How many loading zones?: 1

⛏️ LOADING ZONE 1
Loading Zone ID: L0001
Location ID: L0001
Tonnage/min: 55
Add dumping zones? (y/n): Y
How many dumping zones?: 2

🗑️ DUMPING ZONE 1
Dumping Zone ID: D0002
Location ID: L0002
Tonnage/min: 100

🗑️ DUMPING ZONE 2
Dumping Zone ID: D0003
Location ID: L0003
Tonnage/min: 100
Add gas stations? (y/n): Y
How many GAS stations?: 1

⛽ GAS STATION 1
Gas Station ID: G0004
Location ID: L0004
Service Rate (L/min): 20
Number of Pumps: 5
Add charge